<a href="https://colab.research.google.com/github/aliyyah-u/NLP_Medical_NER/blob/main/NLP_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load & Preprocess Dataset, Tokenisation, Baseline Model

In [1]:
!pip install -q datasets spacy evaluate
!pip install -q transformers torch
!python -m spacy download en_core_web_sm

from datasets import load_dataset
import spacy
import pandas as pd
from collections import Counter
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, DataCollatorForTokenClassification
import evaluate
import numpy as np
import os
os.environ["WANDB_DISABLED"] = "true"

ds = load_dataset("rjac/biobert-ner-diseases-dataset")
print("\nInitial dataset:",ds)

# extract label names from tag values
label_list = ds["train"].features["tags"].feature.names

# NER tag distribution in training set
all_labels = [tag for seq in ds["train"]["tags"] for tag in seq]
label_counts = Counter(all_labels)
print("\nNER tag distribution (train):")
for i, count in label_counts.items():
    print(label_list[i]+ ":" + str(count))

# preprocessing (POS tagging)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner", "lemmatizer"])

def add_pos_tags(batch):
    texts = [" ".join(tokens) for tokens in batch["tokens"]]
    batch["pos_tags"] = []
    for doc in nlp.pipe(texts):
        batch["pos_tags"].append([token.pos_ for token in doc])
    return batch

ds = ds.map(add_pos_tags, batched=True)

all_pos_tags = [tag for sublist in ds["train"]["pos_tags"] for tag in sublist]
pos_counts = Counter(all_pos_tags)
print("\nPOS Tag Distribution in training dataset:")
print(pos_counts.most_common())

for tag in set(all_pos_tags):
    print(tag + ": " + str(spacy.explain(tag)))

print("Dataset sample:")
print({
    "tokens": ds["train"][0]["tokens"][:5],
    "tags": ds["train"][0]["tags"][:5],
    "pos_tags": ds["train"][0]["pos_tags"][:5]
})

# Tokenisation
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, padding=True, is_split_into_words=True)
    labels = []

    for i, mylabel in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(mylabel[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_ds = ds.map(tokenize_and_align_labels, batched=True)

# baseline model
ds["test"] = ds["test"].add_column("baseline_pred", [[0]*len(tokens) for tokens in ds["test"]["tokens"]])

print("\nBaseline sample predictions:")
sample_df = pd.DataFrame({
    "Token": ds["test"]["tokens"][0],
    "POS tag": ds["test"]["pos_tags"][0],
    "True": [label_list[t] for t in ds["test"]["tags"][0]],
    "Pred": [label_list[p] for p in ds["test"]["baseline_pred"][0]]
}).head(20)
print(sample_df)

print("\nBaseline Classification Report:")
true_tags = [t for tags in ds["test"]["tags"] for t in tags]
pred_tags = [p for pred in ds["test"]["baseline_pred"] for p in pred]
print(classification_report(true_tags, pred_tags, target_names=label_list, zero_division=0, digits=4))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/649 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

(…)-00000-of-00001-baac38b53532b0da.parquet:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/15488 [00:00<?, ? examples/s]

Map:   0%|          | 0/5737 [00:00<?, ? examples/s]


Baseline sample predictions:
             Token POS tag       True Pred
0          Torsade   PROPN  B-Disease    O
1               de       X  I-Disease    O
2          pointes    NOUN  I-Disease    O
3      ventricular     ADJ  B-Disease    O
4      tachycardia    NOUN  I-Disease    O
5           during     ADP          O    O
6              low     ADJ          O    O
7             dose    NOUN          O    O
8     intermittent     ADJ          O    O
9       dobutamine    NOUN          O    O
10       treatment    NOUN          O    O
11              in     ADP          O    O
12               a     DET          O    O
13         patient    NOUN          O    O
14            with     ADP          O    O
15         dilated    VERB  B-Disease    O
16  cardiomyopathy     ADJ  I-Disease    O
17             and   CCONJ          O    O
18      congestive     ADJ  B-Disease    O
19           heart    NOUN  I-Disease    O

Baseline Classification Report:
              precision    recall 

# Simple CRF Model

In [2]:
!pip install -q sklearn-crfsuite

import sklearn_crfsuite
from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics

# My feature dictionary for all tokens
def extract_token_features(token, pos):
    return {
        "word.lower()": token.lower(),
        "word.isupper()": token.isupper(),
        "word.istitle()": token.istitle(),
        "word.isdigit()": token.isdigit(),
        "postag": pos,
    }

def sent_to_features(tokens, pos_tags):
    return [extract_token_features(tok, pos) for tok, pos in zip(tokens, pos_tags)]

# method to extract label names from tag values
def sent_to_labels(tag_ids):
    return [label_list[tag] for tag in tag_ids]

# prepare data
X_train = [sent_to_features(tokens, pos_tags) for tokens, pos_tags in zip(ds["train"]["tokens"], ds["train"]["pos_tags"])]
y_train = [sent_to_labels(tags) for tags in ds["train"]["tags"]] # convert id2str

X_test = [sent_to_features(tokens, pos_tags) for tokens, pos_tags in zip(ds["test"]["tokens"], ds["test"]["pos_tags"])]
y_test = [sent_to_labels(tags) for tags in ds["test"]["tags"]]

# CRF model
crf = CRF()
crf.fit(X_train, y_train)
y_pred = crf.predict(X_test)

print("\nCRF Classification Report:")
print(metrics.flat_classification_report(y_test, y_pred, digits=4))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.4 MB/s eta 0:00:00

CRF Classification Report:
              precision    recall  f1-score   support

   B-Disease     0.8389    0.5318    0.6509      5384
   I-Disease     0.7639    0.6185    0.6835      3824
           O     0.9756    0.9945    0.9850    140039

    accuracy                         0.9681    149247
   macro avg     0.8595    0.7149    0.7731    149247
weighted avg     0.9653    0.9681    0.9652    149247



# DistilBERT Model

In [3]:
from transformers import DistilBertForTokenClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
import numpy as np
import torch

# use GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load model and tokenizer
model = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_list)).to(device)
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

data_collator = DataCollatorForTokenClassification(tokenizer)

train_dataset = tokenized_ds["train"]
eval_dataset = tokenized_ds["test"]

training_args = TrainingArguments(
    output_dir="./ner_output",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator = data_collator
)

trainer.train()

results = trainer.evaluate()
print("Eval Results:", results)

predictions, label_ids, _ = trainer.predict(eval_dataset)
pred_labels = np.argmax(predictions, axis=2)

true_labels = [[label_list[l] for l in label_ids[i]] for i in range(len(label_ids))]
pred_labels = [[label_list[l] for l in pred_labels[i]] for i in range(len(pred_labels))]

# flat_true = [item for sublist in true_labels for item in sublist]
# flat_pred = [item for sublist in pred_labels for item in sublist]
# print(classification_report(flat_true, flat_pred, digits=4))

flat_true = []
flat_pred = []

for i in range(len(label_ids)):
    for j in range(len(label_ids[i])):
        if label_ids[i][j] != -100:
            flat_true.append(label_list[label_ids[i][j]])
            flat_pred.append(pred_labels[i][j])

print(classification_report(flat_true, flat_pred, digits=4))

# save model
from google.colab import drive
drive.mount('/content/drive')

save_path = "/content/drive/My Drive/Colab Notebooks/CW_MedNER_UG/distilbert_ner_model"
os.makedirs(save_path, exist_ok=True)
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print("Model saved to:" + str(save_path))

# uncomment code below to load model:
# from transformers import DistilBertForTokenClassification, DistilBertTokenizerFast
# model = DistilBertForTokenClassification.from_pretrained(save_path)
# tokenizer = DistilBertTokenizerFast.from_pretrained(save_path)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-3-7b9fea218060>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.056400,0.058843
2,0.031700,0.059003
3,0.016200,0.065679


Eval Results: {'eval_loss': 0.06567850708961487, 'eval_runtime': 24.8559, 'eval_samples_per_second': 230.81, 'eval_steps_per_second': 14.443, 'epoch': 3.0}


IndexError: list index out of range